<a href="https://colab.research.google.com/github/Ritesh24f1001677/Net_Zero-Maturity-Model-IIT-BHU-INTERNSHIP/blob/main/Dataset_NZE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import pandas as pd
df=pd.read_csv(
    "/content/8b68ae56-84cf-4728-a0a6-1be11028dea7_f27ed1ded4be837214e4603e68bf2c52.csv",
    engine="python",
    on_bad_lines="skip"
)
df.head()


,LG_ST_Code,State,LG_DT_Code,District,Pincode,RegistrationDate,EnterpriseName,CommunicationAddress,Activities
0,27,MAHARASHTRA,490,PUNE,412219.0,04/05/2024,YASH TRANSPORT,"0, NEAR BHANOBA MANDIR, KUSEGAON SUPA ROAD, 0,...","[{""NIC5DigitId"":""49231"",""Description"":""Motoris..."
1,27,MAHARASHTRA,490,PUNE,411024.0,04/05/2024,SHREE SWAMI SAMARTH ENTERPRISES,"., NEAR HANUMAN TEMPLE, SANASWADI RAOD, ., HAV...","[{""NIC5DigitId"":""23929"",""Description"":""Manufac..."
2,27,MAHARASHTRA,490,PUNE,411039.0,17/04/2024,ATHARV AUTO ELECTRICALS,"SURVEY NO. 19, DHAWADE WASTI ,LANDGE WASTI, NE...","[{""NIC5DigitId"":""71200"",""Description"":""Technic..."
3,27,MAHARASHTRA,490,PUNE,411041.0,17/04/2024,RKH ENGINEERING,"FL NO 104, S NO 128/5/2, Bhawantak Resi Near B...","[{""NIC5DigitId"":""41003"",""Description"":""Assembl..."
4,27,MAHARASHTRA,490,PUNE,412114.0,28/03/2024,VIVA ENTERPRISES,"VIVA ENTERPRISES, GAT NO 90, NEAR MADHUBAN STE...","[{""NIC5DigitId"":""96010"",""Description"":""Washing..."


In [9]:
import pandas as pd
import json

# ==================================================
# STEP 1: NIC → ACTIVITY TYPE MAPPING (RULE-SET A)
# ==================================================

SERVICE_NIC_PREFIXES = (
    "49",  # Transport
    "50",  # Water transport
    "51",  # Air transport
    "52",  # Warehousing & logistics
    "53",  # Postal & courier
    "58", "59", "60", "61", "62", "63",  # IT & knowledge services
    "69", "70", "71", "72", "73", "74",  # Professional / technical services
    "95", "96"  # Repair, personal & household services
)

LIGHT_MANUFACTURING_NIC_PREFIXES = (
    "10", "11", "12",  # Food processing
    "13", "14", "15",  # Textiles & garments
    "16", "17", "18",  # Wood, paper, printing
    "22", "23",        # Rubber, plastics, clay
    "31", "32",        # Furniture, light assembly
    "41"               # Construction assembly / prefab erection
)

HEAVY_MANUFACTURING_NIC_PREFIXES = (
    "20", "21",                   # Chemicals & pharmaceuticals
    "24", "25", "26", "27", "28",  # Metals & machinery
    "29", "30",                   # Auto & transport equipment
    "33"                           # Heavy repair & installation
)

# ==================================================
# STEP 2: ACTIVITY TYPE → CAPITAL INTENSITY
# ==================================================

ACTIVITY_TO_CAPITAL = {
    "Service": "Low",
    "Light_Manufacturing": "Medium",
    "Heavy_Manufacturing": "High"
}

# ==================================================
# STEP 3: CAPITAL INTENSITY → MSME PROXY CLASS
# ==================================================

CAPITAL_TO_MSME = {
    "Low": "Micro",
    "Medium": "Small",
    "High": "Medium"
}

CAPITAL_INTENSITY_ORDER = {
    "Low": 1,
    "Medium": 2,
    "High": 3
}

# ==================================================
# STEP 4: HELPER FUNCTIONS
# ==================================================

def classify_activity_type(nic_code):
    """
    Classify NIC code into activity type using prefix logic
    """
    if not nic_code or not isinstance(nic_code, str):
        return "Service"

    for prefix in HEAVY_MANUFACTURING_NIC_PREFIXES:
        if nic_code.startswith(prefix):
            return "Heavy_Manufacturing"

    for prefix in LIGHT_MANUFACTURING_NIC_PREFIXES:
        if nic_code.startswith(prefix):
            return "Light_Manufacturing"

    for prefix in SERVICE_NIC_PREFIXES:
        if nic_code.startswith(prefix):
            return "Service"

    # Conservative default
    return "Service"


def extract_capital_intensity(activities_cell):
    """
    Handles multiple NICs:
    Final capital intensity = MAX intensity among NICs
    """
    if pd.isna(activities_cell):
        return "Low"

    try:
        # Parse JSON safely
        activities = json.loads(activities_cell)

        if not isinstance(activities, list):
            return "Low"

        max_intensity_value = 1  # Default = Low

        for activity in activities:
            nic_code = str(activity.get("NIC5DigitId", ""))

            activity_type = classify_activity_type(nic_code)
            capital_level = ACTIVITY_TO_CAPITAL.get(activity_type, "Low")

            max_intensity_value = max(
                max_intensity_value,
                CAPITAL_INTENSITY_ORDER[capital_level]
            )

        # Convert back to intensity label
        for level, value in CAPITAL_INTENSITY_ORDER.items():
            if value == max_intensity_value:
                return level

    except Exception:
        return "Low"


def determine_msme_proxy_class(capital_intensity):
    """
    Convert capital intensity → MSME proxy class
    """
    return CAPITAL_TO_MSME.get(capital_intensity, "Micro")

# ==================================================
# STEP 5: APPLY TO DATAFRAME
# ==================================================

# Example:
# df = pd.read_csv("your_dataset.csv")

df["capital_intensity_proxy"] = df["Activities"].apply(extract_capital_intensity)
df["msme_proxy_class"] = df["capital_intensity_proxy"].apply(determine_msme_proxy_class)

# Optional cleanup
df.drop(columns=["capital_intensity_proxy"], inplace=True)

# ==================================================
# DONE ✅
# ==================================================

print("✅ MSME Proxy Classification column created successfully!")
print(df["msme_proxy_class"].value_counts())


✅ MSME Proxy Classification column created successfully!
msme_proxy_class
Micro     642501
Small     162826
Medium     89383
Name: count, dtype: int64


In [10]:
df

,LG_ST_Code,State,LG_DT_Code,District,Pincode,RegistrationDate,EnterpriseName,CommunicationAddress,Activities,msme_proxy_class
0,27,MAHARASHTRA,490,PUNE,412219.0,04/05/2024,YASH TRANSPORT,"0, NEAR BHANOBA MANDIR, KUSEGAON SUPA ROAD, 0,...","[{""NIC5DigitId"":""49231"",""Description"":""Motoris...",Micro
1,27,MAHARASHTRA,490,PUNE,411024.0,04/05/2024,SHREE SWAMI SAMARTH ENTERPRISES,"., NEAR HANUMAN TEMPLE, SANASWADI RAOD, ., HAV...","[{""NIC5DigitId"":""23929"",""Description"":""Manufac...",Small
2,27,MAHARASHTRA,490,PUNE,411039.0,17/04/2024,ATHARV AUTO ELECTRICALS,"SURVEY NO. 19, DHAWADE WASTI ,LANDGE WASTI, NE...","[{""NIC5DigitId"":""71200"",""Description"":""Technic...",Micro
3,27,MAHARASHTRA,490,PUNE,411041.0,17/04/2024,RKH ENGINEERING,"FL NO 104, S NO 128/5/2, Bhawantak Resi Near B...","[{""NIC5DigitId"":""41003"",""Description"":""Assembl...",Small
4,27,MAHARASHTRA,490,PUNE,412114.0,28/03/2024,VIVA ENTERPRISES,"VIVA ENTERPRISES, GAT NO 90, NEAR MADHUBAN STE...","[{""NIC5DigitId"":""96010"",""Description"":""Washing...",Micro
...,...,...,...,...,...,...,...,...,...,...
894705,27,MAHARASHTRA,490,PUNE,411042.0,10/01/2026,LADIES TAILOR,"335Ghorpade pethPune, Gorpade peth, police cho...","[{""NIC5DigitId"":""13996"",""Description"":""Manufac...",Small
894706,27,MAHARASHTRA,490,PUNE,412404.0,10/01/2026,GARGOTE TRACTOR SERVICES,"GARGOTWADI, KADUS, GARGOTWADI, GARGOTWADI, KHE...","[{""NIC5DigitId"":""52219"",""Description"":""Other l...",Micro
894707,27,MAHARASHTRA,490,PUNE,410501.0,10/01/2026,VS & Associates,"., Sandbhor Vasti, Alandi Vadgaon Road, ., Khe...","[{""NIC5DigitId"":""46632"",""Description"":""Wholesa...",Micro
894708,27,MAHARASHTRA,490,PUNE,411004.0,10/01/2026,NO 1 BHURJI & MOMOS CORNER,"54, GHANVAT PATIL BUILDING, KARVE ROAD, 54, KA...","[{""NIC5DigitId"":""10799"",""Description"":""Other s...",Small


Error: Runtime no longer has a reference to this dataframe, please re-run this cell and try again.
Error: Runtime no longer has a reference to this dataframe, please re-run this cell and try again.


In [21]:
import pandas as pd
import json

# ==================================================
# STEP 0: LOAD DATASET
# ==================================================

INPUT_FILE = "/content/8b68ae56-84cf-4728-a0a6-1be11028dea7_f27ed1ded4be837214e4603e68bf2c52.csv"
OUTPUT_FILE = "msme_master_dataset_with_clusters_pune.csv"

df = pd.read_csv(INPUT_FILE)

# ==================================================
# STEP 1: NIC → ACTIVITY TYPE MAPPING
# ==================================================

SERVICE_NIC_PREFIXES = (
    "49", "50", "51", "52", "53",
    "58", "59", "60", "61", "62", "63",
    "69", "70", "71", "72", "73", "74",
    "95", "96"
)

LIGHT_MANUFACTURING_NIC_PREFIXES = (
    "10", "11", "12",
    "13", "14", "15",
    "16", "17", "18",
    "22", "23",
    "31", "32",
    "41"
)

HEAVY_MANUFACTURING_NIC_PREFIXES = (
    "20", "21",
    "24", "25", "26", "27", "28",
    "29", "30",
    "33"
)

ACTIVITY_TO_CAPITAL = {
    "Service": "Low",
    "Light_Manufacturing": "Medium",
    "Heavy_Manufacturing": "High"
}

CAPITAL_TO_MSME = {
    "Low": "Micro",
    "Medium": "Small",
    "High": "Medium"
}

CAPITAL_INTENSITY_ORDER = {
    "Low": 1,
    "Medium": 2,
    "High": 3
}

# ==================================================
# STEP 2: HELPER FUNCTIONS
# ==================================================

def classify_activity_type(nic_code):
    if not nic_code or not isinstance(nic_code, str):
        return "Service"

    for p in HEAVY_MANUFACTURING_NIC_PREFIXES:
        if nic_code.startswith(p):
            return "Heavy_Manufacturing"

    for p in LIGHT_MANUFACTURING_NIC_PREFIXES:
        if nic_code.startswith(p):
            return "Light_Manufacturing"

    for p in SERVICE_NIC_PREFIXES:
        if nic_code.startswith(p):
            return "Service"

    return "Service"


def extract_capital_intensity(activities_cell):
    if pd.isna(activities_cell):
        return "Low"

    try:
        activities = json.loads(activities_cell)
        max_intensity = 1

        for act in activities:
            nic = str(act.get("NIC5DigitId", ""))
            activity_type = classify_activity_type(nic)
            capital = ACTIVITY_TO_CAPITAL[activity_type]
            max_intensity = max(max_intensity, CAPITAL_INTENSITY_ORDER[capital])

        for k, v in CAPITAL_INTENSITY_ORDER.items():
            if v == max_intensity:
                return k

    except Exception:
        return "Low"


def determine_msme_proxy_class(capital_intensity):
    return CAPITAL_TO_MSME.get(capital_intensity, "Micro")


def extract_dominant_nic_group(activities_cell):
    """
    Extracts first 2 digits of dominant NIC
    """
    if pd.isna(activities_cell):
        return "00"

    try:
        activities = json.loads(activities_cell)
        if len(activities) == 0:
            return "00"

        nic_code = str(activities[0].get("NIC5DigitId", ""))
        return nic_code[:2]

    except Exception:
        return "00"

# ==================================================
# STEP 3: MSME PROXY CLASSIFICATION
# ==================================================

df["capital_intensity_proxy"] = df["Activities"].apply(extract_capital_intensity)
df["msme_proxy_class"] = df["capital_intensity_proxy"].apply(determine_msme_proxy_class)
df.drop(columns=["capital_intensity_proxy"], inplace=True)

# ==================================================
# STEP 4: CLUSTER IDENTIFICATION
# ==================================================

df["nic_sector_group"] = df["Activities"].apply(extract_dominant_nic_group)

df["cluster_key"] = (
    df["LG_ST_Code"].astype(str) + "_" +
    df["LG_DT_Code"].astype(str) + "_" +
    df["Pincode"].astype(str) + "_" +
    df["nic_sector_group"] + "_" +
    df["msme_proxy_class"]
)

df["cluster_id"] = df.groupby("cluster_key").ngroup() + 1
df["cluster_id"] = "Cluster_" + df["cluster_id"].astype(str)

# Optional cleanup
df.drop(columns=["cluster_key"], inplace=True)

# ==================================================
# STEP 5: SAVE MASTER DATASET
# ==================================================

df.to_csv(OUTPUT_FILE, index=False)

print("✅ MSME MASTER DATASET CREATED SUCCESSFULLY")
print("📁 Saved file:", OUTPUT_FILE)
print("\nSample cluster distribution:")
print(df["cluster_id"].value_counts().head())


✅ MSME MASTER DATASET CREATED SUCCESSFULLY
📁 Saved file: msme_master_dataset_with_clusters_pune.csv

Sample cluster distribution:
cluster_id
Cluster_9814     2676
Cluster_2257     2670
Cluster_7953     2657
Cluster_2254     2605
Cluster_13052    2595
Name: count, dtype: int64


In [22]:
df

,LG_ST_Code,State,LG_DT_Code,District,Pincode,RegistrationDate,EnterpriseName,CommunicationAddress,Activities,msme_proxy_class,nic_sector_group,cluster_id
0,27,MAHARASHTRA,490,PUNE,412219.0,04/05/2024,YASH TRANSPORT,"0, NEAR BHANOBA MANDIR, KUSEGAON SUPA ROAD, 0,...","[{""NIC5DigitId"":""49231"",""Description"":""Motoris...",Micro,49,Cluster_16285
1,27,MAHARASHTRA,490,PUNE,411024.0,04/05/2024,SHREE SWAMI SAMARTH ENTERPRISES,"., NEAR HANUMAN TEMPLE, SANASWADI RAOD, ., HAV...","[{""NIC5DigitId"":""23929"",""Description"":""Manufac...",Small,23,Cluster_6698
2,27,MAHARASHTRA,490,PUNE,411039.0,17/04/2024,ATHARV AUTO ELECTRICALS,"SURVEY NO. 19, DHAWADE WASTI ,LANDGE WASTI, NE...","[{""NIC5DigitId"":""71200"",""Description"":""Technic...",Micro,71,Cluster_8820
3,27,MAHARASHTRA,490,PUNE,411041.0,17/04/2024,RKH ENGINEERING,"FL NO 104, S NO 128/5/2, Bhawantak Resi Near B...","[{""NIC5DigitId"":""41003"",""Description"":""Assembl...",Small,41,Cluster_9056
4,27,MAHARASHTRA,490,PUNE,412114.0,28/03/2024,VIVA ENTERPRISES,"VIVA ENTERPRISES, GAT NO 90, NEAR MADHUBAN STE...","[{""NIC5DigitId"":""96010"",""Description"":""Washing...",Micro,96,Cluster_13875
...,...,...,...,...,...,...,...,...,...,...,...,...
894705,27,MAHARASHTRA,490,PUNE,411042.0,10/01/2026,LADIES TAILOR,"335Ghorpade pethPune, Gorpade peth, police cho...","[{""NIC5DigitId"":""13996"",""Description"":""Manufac...",Small,13,Cluster_9187
894706,27,MAHARASHTRA,490,PUNE,412404.0,10/01/2026,GARGOTE TRACTOR SERVICES,"GARGOTWADI, KADUS, GARGOTWADI, GARGOTWADI, KHE...","[{""NIC5DigitId"":""52219"",""Description"":""Other l...",Micro,52,Cluster_17810
894707,27,MAHARASHTRA,490,PUNE,410501.0,10/01/2026,VS & Associates,"., Sandbhor Vasti, Alandi Vadgaon Road, ., Khe...","[{""NIC5DigitId"":""46632"",""Description"":""Wholesa...",Micro,46,Cluster_2251
894708,27,MAHARASHTRA,490,PUNE,411004.0,10/01/2026,NO 1 BHURJI & MOMOS CORNER,"54, GHANVAT PATIL BUILDING, KARVE ROAD, 54, KA...","[{""NIC5DigitId"":""10799"",""Description"":""Other s...",Small,10,Cluster_4169


In [15]:
# ==================================================
# STEP 6: NUMBER OF UNIQUE CLUSTERS
# ==================================================

num_clusters = df["cluster_id"].nunique()

print("\n📊 Total number of unique MSME clusters:", num_clusters)



📊 Total number of unique MSME clusters: 24048


In [16]:
df.head()

,LG_ST_Code,State,LG_DT_Code,District,Pincode,RegistrationDate,EnterpriseName,CommunicationAddress,Activities,msme_proxy_class,nic_sector_group,cluster_id
0,27,MAHARASHTRA,490,PUNE,412219.0,04/05/2024,YASH TRANSPORT,"0, NEAR BHANOBA MANDIR, KUSEGAON SUPA ROAD, 0,...","[{""NIC5DigitId"":""49231"",""Description"":""Motoris...",Micro,49,Cluster_16285
1,27,MAHARASHTRA,490,PUNE,411024.0,04/05/2024,SHREE SWAMI SAMARTH ENTERPRISES,"., NEAR HANUMAN TEMPLE, SANASWADI RAOD, ., HAV...","[{""NIC5DigitId"":""23929"",""Description"":""Manufac...",Small,23,Cluster_6698
2,27,MAHARASHTRA,490,PUNE,411039.0,17/04/2024,ATHARV AUTO ELECTRICALS,"SURVEY NO. 19, DHAWADE WASTI ,LANDGE WASTI, NE...","[{""NIC5DigitId"":""71200"",""Description"":""Technic...",Micro,71,Cluster_8820
3,27,MAHARASHTRA,490,PUNE,411041.0,17/04/2024,RKH ENGINEERING,"FL NO 104, S NO 128/5/2, Bhawantak Resi Near B...","[{""NIC5DigitId"":""41003"",""Description"":""Assembl...",Small,41,Cluster_9056
4,27,MAHARASHTRA,490,PUNE,412114.0,28/03/2024,VIVA ENTERPRISES,"VIVA ENTERPRISES, GAT NO 90, NEAR MADHUBAN STE...","[{""NIC5DigitId"":""96010"",""Description"":""Washing...",Micro,96,Cluster_13875


In [17]:
df.shape

(894710, 12)

In [18]:
import pandas as pd
df1=pd.read_csv(
    "/content/8b68ae56-84cf-4728-a0a6-1be11028dea7_96d973ceae41a664629fbbd71245b212.csv",
    engine="python",
    on_bad_lines="skip"
)
df1.head()


,LG_ST_Code,State,LG_DT_Code,District,Pincode,RegistrationDate,EnterpriseName,CommunicationAddress,Activities
0,3,PUNJAB,34,JALANDHAR,144003.0,08/02/2022,SHINETOY AND GIFTS,"NEAR SHIV MANDIR, RED ROSE COLONY, NA, NEAR SH...","[{""NIC5DigitId"":""47640"",""Description"":""Retail ..."
1,3,PUNJAB,34,JALANDHAR,144002.0,11/04/2022,Poonam,"47, New Lasuri Mohalla, .., 47, .., Jalandhar","[{""NIC5DigitId"":""14105"",""Description"":""Custom ..."
2,3,PUNJAB,34,JALANDHAR,144410.0,11/04/2022,ARJUN SINGH,"H.NO.C/121, WARD NO.10, ., H.NO.C/121, PHILLAU...","[{""NIC5DigitId"":""78100"",""Description"":""Activit..."
3,3,PUNJAB,34,JALANDHAR,144001.0,14/06/2022,RAJESH CHAT HOUSE,"KABIR VIHAR, -, -, KABIR VIHAR, -, JALANDHAR","[{""NIC5DigitId"":""56210"",""Description"":""Event c..."
4,3,PUNJAB,34,JALANDHAR,144040.0,13/12/2021,Prabhjot Trading co,"Shop no 40, New Dana mandi, Malsian road, Shop...","[{""NIC5DigitId"":""46101"",""Description"":""Activit..."


In [20]:
import pandas as pd
import json

# ==================================================
# STEP 0: LOAD DATASET
# ==================================================

# 🔴 CHANGE ONLY THIS PATH
INPUT_FILE = "/content/8b68ae56-84cf-4728-a0a6-1be11028dea7_96d973ceae41a664629fbbd71245b212.csv"
OUTPUT_FILE = "msme_master_dataset_with_clusters.csv"

df1 = pd.read_csv(INPUT_FILE)

# ==================================================
# STEP 1: NIC → ACTIVITY TYPE MAPPING
# ==================================================

SERVICE_NIC_PREFIXES = (
    "45", "46", "47",                 # Trade & retail
    "49", "50", "51", "52", "53",     # Transport & logistics
    "55", "56",                       # Hospitality
    "58", "59", "60", "61", "62", "63",  # IT & media
    "69", "70", "71", "72", "73", "74",  # Professional services
    "77", "78", "79",                 # Admin & placement services
    "95", "96"                        # Repair & personal services
)

LIGHT_MANUFACTURING_NIC_PREFIXES = (
    "10", "11", "12",                 # Food
    "13", "14", "15",                 # Textiles & garments
    "16", "17", "18",                 # Wood, paper, printing
    "22", "23",                       # Rubber, plastics, clay
    "31", "32",                       # Furniture & assembly
    "41"                              # Construction / prefab
)

HEAVY_MANUFACTURING_NIC_PREFIXES = (
    "20", "21",                       # Chemicals & pharma
    "24", "25", "26", "27", "28",     # Metals & machinery
    "29", "30",                       # Auto & transport equipment
    "33"                              # Heavy installation & repair
)

# ==================================================
# STEP 2: CAPITAL INTENSITY → MSME PROXY
# ==================================================

ACTIVITY_TO_CAPITAL = {
    "Service": "Low",
    "Light_Manufacturing": "Medium",
    "Heavy_Manufacturing": "High"
}

CAPITAL_TO_MSME = {
    "Low": "Micro",
    "Medium": "Small",
    "High": "Medium"
}

CAPITAL_INTENSITY_ORDER = {
    "Low": 1,
    "Medium": 2,
    "High": 3
}

# ==================================================
# STEP 3: HELPER FUNCTIONS
# ==================================================

def classify_activity_type(nic_code):
    if not nic_code or not isinstance(nic_code, str):
        return "Service"

    for p in HEAVY_MANUFACTURING_NIC_PREFIXES:
        if nic_code.startswith(p):
            return "Heavy_Manufacturing"

    for p in LIGHT_MANUFACTURING_NIC_PREFIXES:
        if nic_code.startswith(p):
            return "Light_Manufacturing"

    for p in SERVICE_NIC_PREFIXES:
        if nic_code.startswith(p):
            return "Service"

    return "Service"


def extract_capital_intensity(activities_cell):
    if pd.isna(activities_cell):
        return "Low"

    try:
        activities = json.loads(activities_cell)
        max_intensity = 1

        for act in activities:
            nic = str(act.get("NIC5DigitId", ""))
            activity_type = classify_activity_type(nic)
            capital = ACTIVITY_TO_CAPITAL[activity_type]
            max_intensity = max(max_intensity, CAPITAL_INTENSITY_ORDER[capital])

        for k, v in CAPITAL_INTENSITY_ORDER.items():
            if v == max_intensity:
                return k

    except Exception:
        return "Low"


def determine_msme_proxy_class(capital_intensity):
    return CAPITAL_TO_MSME.get(capital_intensity, "Micro")


def extract_dominant_nic_group(activities_cell):
    if pd.isna(activities_cell):
        return "00"

    try:
        activities = json.loads(activities_cell)
        if not activities:
            return "00"

        nic_code = str(activities[0].get("NIC5DigitId", ""))
        return nic_code[:2]

    except Exception:
        return "00"

# ==================================================
# STEP 4: MSME PROXY CLASSIFICATION
# ==================================================

df1["capital_intensity_proxy"] = df1["Activities"].apply(extract_capital_intensity)
df1["msme_proxy_class"] = df1["capital_intensity_proxy"].apply(determine_msme_proxy_class)
df1.drop(columns=["capital_intensity_proxy"], inplace=True)

# ==================================================
# STEP 5: CLUSTER IDENTIFICATION (SPATIAL + FUNCTIONAL)
# ==================================================

df1["nic_sector_group"] = df1["Activities"].apply(extract_dominant_nic_group)

df1["cluster_key"] = (
    df1["LG_ST_Code"].astype(str) + "_" +
    df1["LG_DT_Code"].astype(str) + "_" +
    df1["Pincode"].astype(str) + "_" +
    df1["nic_sector_group"] + "_" +
    df1["msme_proxy_class"]
)

df1["cluster_id"] = df1.groupby("cluster_key").ngroup() + 1
df1["cluster_id"] = "Cluster_" + df1["cluster_id"].astype(str)

df1.drop(columns=["cluster_key"], inplace=True)

# ==================================================
# STEP 6: SAVE MASTER DATASET
# ==================================================

df1.to_csv(OUTPUT_FILE, index=False)

# ==================================================
# STEP 7: REPORTING
# ==================================================

print("✅ MSME MASTER DATASET CREATED SUCCESSFULLY")
print("📁 Output file:", OUTPUT_FILE)

print("\n📊 MSME Distribution:")
print(df1["msme_proxy_class"].value_counts())

print("\n📊 Total number of unique clusters:")
print(df1["cluster_id"].nunique())

print("\n🔝 Top 10 clusters by size:")
print(df1["cluster_id"].value_counts().head(10))


✅ MSME MASTER DATASET CREATED SUCCESSFULLY
📁 Output file: msme_master_dataset_with_clusters.csv

📊 MSME Distribution:
msme_proxy_class
Micro     91822
Small     31882
Medium     9188
Name: count, dtype: int64

📊 Total number of unique clusters:
6396

🔝 Top 10 clusters by size:
cluster_id
Cluster_521     4654
Cluster_518     2781
Cluster_665     1548
Cluster_904     1374
Cluster_610     1331
Cluster_789     1250
Cluster_536     1141
Cluster_647     1073
Cluster_499     1059
Cluster_1285    1008
Name: count, dtype: int64
